# Tile Arithmetic
Rasterframes includes support for arithmetic operations across tiles, calling the function on the cells that correspond to each other and producing the result of these operations as a tile. For instance, given a tile with pixels that are all 1's and a tile whose pixels are all 2's, the result of a localAdd on those tiles would be a tile with pixel values all set to 3. This example uses `localAdd`, but `localSubtract`, `localDivide`, and `localMultiply` also exist. While not used in this example, `localAddScalar`, `localSubtractScalar`, etc can be uses to perform operations with a scalar, as opposed to the local values in another tile. 

Initial configs

In [1]:
import astraea.spark.rasterframes._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import org.apache.spark.sql._


implicit val spark = SparkSession.builder().
  master("local").appName("RasterFrames").
  config("spark.ui.enabled", "false").
  getOrCreate().
  withRasterFrames

def readTiff(name: String): SinglebandGeoTiff = SinglebandGeoTiff(s"../samples/$name")

Intitializing Scala interpreter ...

Spark Web UI available at http://172.18.0.2:4042
SparkContext available as 'sc' (version = 2.2.0, master = local[*], app id = local-1531152848439)
SparkSession available as 'spark'


import astraea.spark.rasterframes._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import org.apache.spark.sql._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4ee7b2b
readTiff: (name: String)geotrellis.raster.io.geotiff.SinglebandGeoTiff


In [2]:
val filenamePattern = "L8-B%d-Elkton-VA.tiff"
val bandNumbers = 1 to 4
val bandColNames = bandNumbers.map(b ⇒ s"band_$b").toArray

filenamePattern: String = L8-B%d-Elkton-VA.tiff
bandNumbers: scala.collection.immutable.Range.Inclusive = Range(1, 2, 3, 4)
bandColNames: Array[String] = Array(band_1, band_2, band_3, band_4)


Once imported, join the tiles together based on a `spatialKey`.

In [3]:
val joinedRF = bandNumbers.
  map { b ⇒ (b, filenamePattern.format(b)) }.
  map { case (b, f) ⇒ (b, readTiff(f)) }.
  map { case (b, t) ⇒ t.projectedRaster.toRF(s"band_$b") }.
  reduce(_ spatialJoin _)

joinedRF: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, band_1: rf_tile ... 3 more fields]


New columns are added based on the results of these local functions. `localAdd` adds corresponding cells, `localDivide` divides them, `localSubtractScalar` subtracts a scalar from every cell in the input tile, and `localMultiplyScalar` multiplies each cell by a scalar value.

In [4]:
val addRF = joinedRF.withColumn("B1+B2", localAdd($"band_1", $"band_2")).asRF
val divideRF = joinedRF.withColumn("B1/B2", localDivide($"band_1", $"band_2")).asRF
val subScalarRF = joinedRF.withColumn("B1-100", localSubtractScalar($"band_1", 100)).asRF

addRF: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, band_1: rf_tile ... 4 more fields]
divideRF: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, band_1: rf_tile ... 4 more fields]
subScalarRF: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, band_1: rf_tile ... 4 more fields]


In [11]:
val subConstantRF = joinedRF.withColumn("B1 - 6", localSubtractScalar($"band_1", 6))
val multScalarRF = joinedRF.withColumn("B1*2", localMultiplyScalar($"band_1", 2)).asRF

subConstantRF: org.apache.spark.sql.DataFrame = [spatial_key: struct<col: int, row: int>, band_1: rf_tile ... 4 more fields]
multScalarRF: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, band_1: rf_tile ... 4 more fields]


`tileMax` computes the max values of the cells in a tile, `tileSum` computes the sum of all cells, `tileMean` finds the mean value, and `tileMin` finds the minimum value.

In [6]:
addRF.select(tileMax($"B1+B2"), tileMax($"band_1"), tileMax($"band_2")).show()

+--------------+---------------+---------------+
|tileMax(B1+B2)|tileMax(band_1)|tileMax(band_2)|
+--------------+---------------+---------------+
|       42925.0|        20778.0|        22147.0|
+--------------+---------------+---------------+



In [7]:
divideRF.select(tileSum($"B1/B2"), tileSum($"band_1"), tileSum($"band_2")).show()

+--------------+---------------+---------------+
|tileSum(B1/B2)|tileSum(band_1)|tileSum(band_2)|
+--------------+---------------+---------------+
|       31389.0|   3.09149454E8|     2.806676E8|
+--------------+---------------+---------------+



In [8]:
subScalarRF.select(tileMean($"B1-100"), tileMean($"band_1")).show()

+-----------------+-----------------+
| tileMean(B1-100)| tileMean(band_1)|
+-----------------+-----------------+
|9734.874785264363|9834.874785264363|
+-----------------+-----------------+



In [9]:
multScalarRF.select(tileMin($"B1*2"), tileMin($"band_1")).show()

+-------------+---------------+
|tileMin(B1*2)|tileMin(band_1)|
+-------------+---------------+
|      18522.0|         9261.0|
+-------------+---------------+



In [12]:
subConstantRF.select(tileMin($"B1 - 6"), tileMin($"band_1")).show()

+---------------+---------------+
|tileMin(B1 - C)|tileMin(band_1)|
+---------------+---------------+
|         9255.0|         9261.0|
+---------------+---------------+

